# Sampling the data and random walking

We will simulate interaction and information exchange through random walking 

In [ ]:
# Message gossip
from p2psimpy import *
from p2psimpy.services.base import BaseHandler
from p2psimpy.messages import BaseMessage
from p2psimpy.storage import Storage

from p2psimpy.services.base import BaseRunner

from re import split 
from copy import copy

from collections import defaultdict 

import random
import networkx as nx
from trust import RandomWalks



# Define a special message GossipMessage: Message with ttl
class Transaction(BaseMessage):
    size = 300
    
        
class TransactionService(BaseHandler):
    """
    A service to respond for transactions (a signed interaction between two parties) 
    """

    def __init__(self, peer):
        super().__init__(peer)
        
        self.strg_name = 'tx_val'
        self.time_name = 'tx_time'
        
        strg = self.peer.get_storage(self.strg_name)
        if not strg:
            self.peer.add_storage(self.strg_name, Storage())
        
        self.peer.add_storage(self.time_name, Storage())
        

    def handle_message(self, msg):
        # If the message is to you - sign it and send it back
        msg_id = msg.data['id']
        o_id, p_id = msg_id.split('_') 
        
        strg = self.peer.get_storage(self.strg_name)
        last_val_msg = strg.get(msg_id)
        last_val = 0.0 if not last_val_msg else last_val_msg['val']

        if p_id == str(self.peer.peer_id):
            
            # Check if the transaction is the up to date
            if msg.data['val'] > last_val:
                new_msg_data = msg.data.copy()
                new_msg_data['sign2'] = True
            else: 
                # send old data
                new_msg_data = last_val_msg
                
            # Send the message to the partner 
            new_msg = Transaction(self.peer, new_msg_data)
            partner = msg.sender
            self.peer.send(partner, new_msg)
            self.peer.store(self.strg_name, msg_id, new_msg_data, rewrite=True)
            self.peer.store(self.time_name, msg_id, self.peer.env.now, rewrite=True)
        else:  
            # Store the message if it is the last version             
            if msg.data['val'] > last_val or (msg.data['val'] == last_val and 'sign2' in msg.data):
                # Store new message 
                self.peer.store(self.strg_name, msg_id, msg.data, rewrite=True)        
                self.peer.store(self.time_name, msg_id, self.peer.env.now, rewrite=True)        
        
    @property
    def messages(self):
        return Transaction,



class TransactionProducer(BaseRunner):

    def __init__(self, peer, init_timeout=1000, msg_rate=4):
        '''
        init_timeout: milliseconds to wait before starting the message production. 
        msg_rate: number of messages per second
        '''
        super().__init__(peer)

        # calculate tx_interval
        self.init_timeout = init_timeout
        
        self.tx_interval = 1000 / msg_rate
        self.counter = defaultdict(int)
        
        # Let's add a storage layer to store messages
        self.strg_name = 'tx_val'
        strg = self.peer.get_storage(self.strg_name)
        if not strg: 
            self.peer.add_storage(self.strg_name, Storage())
        self.online = True


    def interact_with_peer(self):
        '''Interact with a random peer in the neighbourhood. '''
        
        # 1. Choose an interaction partner.  
        if not self.online:
            return None
        connected = list(self.peer._get_connections())
        partner = random.choice(connected)
        
        interaction_id = str(self.peer.peer_id) + '_' + str(partner.peer_id)

        strg = self.peer.get_storage(self.strg_name)
        last_tx = strg.get(interaction_id)
        last_count = 0 if not last_tx else last_tx['counter']
        last_val = 0.0 if not last_tx else last_tx['val']
        
        
        #print(interaction_id, last_val)
        # 2. Decide number of resources to request, form the message 
        res = random.randint(1, 100)
        
        msg_data = {'id': interaction_id, 
                    'val': last_val + res, 
                    'counter': last_count + 1, 
                    'sign1': True}
        #print(msg_data)
        if not msg_data:
            print('MSG data is None when producing')
        msg = Transaction(self.peer, msg_data)
        
        # 3. Send the message to the partner 
        self.peer.send(partner, msg)
        self.peer.store(self.strg_name, interaction_id, msg_data, rewrite=True)
        

    def run(self):
        # Wait the initial timeout
        yield self.env.timeout(self.init_timeout)
        while True:
            self.interact_with_peer()
            yield self.env.timeout(self.tx_interval)
            


In [ ]:
class CrawlRequest(BaseMessage):
    size = 50

class CrawlerService(BaseRunner):
    
    def __init__(self, peer, init_timeout=1500, msg_rate=4):
        '''
        init_timeout: milliseconds to wait before starting the message production. 
        msg_rate: number of messages per second
        '''
        super().__init__(peer)

        # calculate tx_interval
        self.init_timeout = init_timeout
        
        self.tx_interval = 1000 / msg_rate
        self.counter = defaultdict(int)
        
        # Let's add a storage layer to store messages
        self.strg_name = 'tx_val'
        self.online = True
        
        # Init graph
        self.graph = nx.DiGraph()
        self.back = False
        

    def crawl_step(self):
        '''Make a crawl step given the local knowledge of the graph '''
        
        # 0. Construct/update the work graph
        strg = self.peer.get_storage(self.strg_name)
        for t in strg.txs.values():
            a, b = t['id'].split('_')
            self.graph.add_edge(a, b, weight = t['val'])
            
        # 1. Take a random walk in the graph
        if not self.online:
            return None
        r = RandomWalks(self.graph)
        walk = r.run_one_walk(str(self.peer.peer_id), reset_probability=0.13, back_random_walk=self.back)
        self.back = not self.back
        
        # 2. Send crawl request to the peer        
        if len(walk) > 1: 
            crawl_peer = walk[1]  
            topic = walk[-1]
            
            # Form a message 
            partner = self.peer.get_connected_peer(crawl_peer)
            if partner: 
                msg = CrawlRequest(self.peer, topic)
                self.peer.send(partner, msg)
        

    def run(self):
        # Wait the initial timeout
        yield self.env.timeout(self.init_timeout)
        while True:
            self.crawl_step()
            yield self.env.timeout(self.tx_interval)
            
            
class CrawlerHandler(BaseHandler):
    
    def __init__(self, peer):
        super().__init__(peer)
        
        self.strg_name = 'tx_val'        
        

    def handle_message(self, msg):
        # If the message is to you - sign it and send it back
        crawl_topic = msg.data
        
        # Send request transactions 
        strg = self.peer.get_storage(self.strg_name)
        for t in strg.txs.values():
            a, b = t['id'].split('_')
            if a == str(crawl_topic) or b == str(crawl_topic):
                req_msg = Transaction(self.peer, t)
                self.peer.send(msg.sender, req_msg)
        
    @property
    def messages(self):
        return CrawlRequest,
    
    
        

### Prepare experiment

In [ ]:
# Define locations 
from p2psimpy.config import *
from p2psimpy.consts import *
from p2psimpy.services.connection_manager import BaseConnectionManager
import networkx as nx
from random import choice

import matplotlib.pyplot as plt

# We take the locations from AWS 
class Locations(Config):
    locations = ['Ohio', 'Ireland', 'Tokyo']
    latencies = {
        'Ohio': {'Ohio': Dist('invgamma', (5.54090, 0.333305, 0.987249)),
                 'Ireland': Dist('norm', (73.6995, 1.19583092197097127)),
                 'Tokyo': Dist('norm', (156.00904977375566, 0.09469886668079797))
                },
        'Ireland':{'Ireland': Dist('invgamma', (6.4360455224301525, 0.8312748033308526, 1.086191852963273)),
                   'Tokyo': Dist('norm', (131.0275, 0.25834811785650774))
                  },
        'Tokyo': {'Tokyo':  Dist('invgamma', (11.104508341331055, 0.3371934865734555, 2.0258998705983737))}
    }
    
# Define peer     
class PeerConfig(Config):
    location = Dist('sample', Locations.locations)
    bandwidth_ul = Dist( 'norm', (50*MBit, 10*MBit))
    bandwidth_dl = Dist( 'norm', (50*MBit, 10*MBit))

# Configuration used for our GossipService
class GossipConfig(Config):
    exclude_types={'client',}

# We have not two types of nodes: *peer* and *client*
def prepare_peer_types():
    return { 'peer': PeerType(PeerConfig, (BaseConnectionManager, 
                                           TransactionService, 
                                           TransactionProducer,
                                           CrawlerService,
                                           CrawlerHandler
                                          ))}

def prepare_topology(num_peers=100, num_clients=0):    
    # Create network topology
    G = nx.erdos_renyi_graph(num_peers, 0.1)   
    nx.relabel_nodes(G, {k: k+1 for k in G.nodes()} ,copy=False)
    
    # Connect the client node to a random peer
    client_edges = [(i, choice(list(G.nodes()))) for i in range(num_peers+1, num_clients+num_peers+1)]
    G.add_edges_from(client_edges)

    types_map = {k: 'peer' if k < num_peers+1 else 'client' for k in G.nodes()}
    # Assign a peer type to the peers 
    nx.set_node_attributes(G, types_map , 'type')
    return G

def visualize_peer_client_network(G):
    plt.figure(figsize=(10,10))

    # Draw client/ peer network 

    master_nodes = [n for (n,ty) in \
        nx.get_node_attributes(G,'type').items() if ty == 'peer']
    client_nodes = [n for (n,ty) in \
        nx.get_node_attributes(G,'type').items() if ty == 'client']

    pos = nx.kamada_kawai_layout(G)

    nx.draw_networkx_nodes(G, pos, nodelist=master_nodes, \
        node_color='blue', node_shape='o', node_size=500)
    nx.draw_networkx_nodes(G, pos, nodelist=client_nodes,  \
        node_color='green', node_shape='^', node_size=100, label=1)

    nx.draw_networkx_labels(G, pos, labels={k:k for k in master_nodes}, font_color='w')

    nx.draw_networkx_edges(G, pos, edgelist=G.subgraph(master_nodes).edges(), width=1.5)
    nx.draw_networkx_edges(G, pos, edgelist=G.edges(nbunch=client_nodes),  style='dotted')



In [ ]:
G = prepare_topology()
visualize_peer_client_network(G)

In [ ]:
from time import time
from p2psimpy.simulation import BaseSimulation

from trust import RandomWalks

net_sim = BaseSimulation(Locations, G, prepare_peer_types())

In [ ]:
net_sim.run(10_200)

In [ ]:
G = nx.DiGraph()
for t in net_sim.peers[10].storage['tx_val'].txs.values():
    a,b = t['id'].split('_')
    G.add_edge(a, b, weight=t['val'])
    

In [ ]:
nx.draw_kamada_kawai(G)

In [ ]:
print(len(G.nodes()))
print(len(G.edges()))


## Big Tech alternative

- Ranking is personal. Clients collect the data, sample the network and output the ranking.
- Clients try to find the top node and sample from the network to build up the reputation mechanism. 

- The client has local interactions with other peers/clients and forms a world assumption. 
- Using the local interactions -> It builds up other ranking mechanisms. 

------

### Assumptions on application

1. **Trust anchors**. There are initial trust anchors - events that can be verified and that are trusted. Possible assumptions: 
 - Minimal: trust only local interactions 
 - Additional trust anchors
 - Maximal: trust any interaction?

2. **Service environment**. How clients and service interact with each other. Can we build a network of interactions. Possible options:
 - Client-Service Providers. Clients don't interact with each other. Service providers don't interact with each other.
 - Service-Service Providers. Everyone is client and everyone is service provider. File-sharing in P2P. 
 - ??? 
 
## Applications 

### Good Marketplaces 

- Asymmetric. Sellers and buyer have different interaction models. 
- Price and money plays a big goal. Sellers are profit-drive, buyer want to maximize the output from the product. 
- Ranking plays a big role, as good rank increases the profts too. Thus sellers have incentives to cheat the ranking.
- The interaction graph is not connected. Might be even bipartile. 

### Service providers marketplaces: Uber Drive, Delivery etc.

- The interaction 


------

## Datasets from the wild

1. Amazon co-purchasing of products. How products are
2. Collaboration network in science. Citation network in science.  
3. Email communication network 
4. Facebook ego-network. Frends-to-frends. 
5. Gnutella peer-to-peer network. 
6. Wikipedia trust










